In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('scraping-371606-9e78aba82a03.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#date = 0 #当日は0、前日は1・・・
#
date = 1 #当日は0、前日は1・・・
date2 = "08/18"
pmc = 24019003 # 店舗ID
SPREADSHEET_KEY = '1ch9Ckr3IiaMb8JLWNc-PWMEE59l4LbkU6ydfPI6mLAA' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 745 # スロットの1台目
wait = 0.02

user_id = "tomosvtg.1988+456@gmail.com"
user_pw = "paruparu22"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
path = "C:\\Users\\garden026\\WD\\" + str(wd) + "\\chromedriver.exe"

chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "200":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "200":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S2RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/18', '025817', '1', '36154.0', '-1504.0', '86', '16']
['08/18', '025817', '2', '8205.0', '-2707.0', '10', '4']
['08/18', '025817', '3', '18462.0', '-3911.0', '35', '8']
['08/18', '025817', '4', '14359.0', '-8124.0', '12', '1']
['08/18', '025817', '5', '17949.0', '-2707.0', '34', '9']
['08/18', '025817', '6', '18718.0', '-8425.0', '18', '7']
['08/18', '025521', '7', '21026.0', '-902.0', '20', '12']
['08/18', '025521', '8', '31795.0', '2709.0', '38', '17']
['08/18', '025521', '9', '23077.0', '302.0', '22', '11']
['08/18', '025521', '10', '23077.0', '-6319.0', '16', '9']
['08/18', '025877', '11', '24615.0', '302.0', '28', '12']
['08/18', '025877', '12', '46410.0', '-902.0', '45', '19']
['08/18', '025877', '13', '37692.0', '-902.0', '40', '17']
['08/18', '025877', '14', '32051.0', '5117.0', '39', '15']
['08/18', '025877', '15', '44359.0', '1205.0', '48', '16']
['08/18', '025877', '16', '9744.0', '1205.0', '12', '3']
['08/18', '025877', '17', '51538.0', '-10231.0', '39', '19']
['08/18'

['08/18', '025829', '142', '4615.0', '6320.0', '7', '1']
['08/18', '025829', '143', '24615.0', '-10832.0', '7', '3']
['08/18', '025829', '144', '9231.0', '-2106.0', '4', '2']
['08/18', '025885', '145', '9231.0', '-4513.0', '16', '1']
['08/18', '025885', '146', '21026.0', '-7823.0', '38', '7']
['08/18', '025885', '147', '10256.0', '-6018.0', '13', '4']
['08/18', '025544', '148', '5897.0', '-2106.0', '9', '4']
['08/18', '025544', '149', '11795.0', '-7522.0', '6', '3']
['08/18', '025544', '150', '6410.0', '6922.0', '39', '2']
['08/18', '025781', '151', '4615.0', '5719.0', '9', '1']
['08/18', '025781', '152', '4872.0', '1205.0', '6', '2']
['08/18', '025739', '153', '5897.0', '-4212.0', '3', '3']
['08/18', '025739', '154', '5897.0', '6320.0', '15', '8']
['08/18', '025739', '155', '11282.0', '-7823.0', '7', '7']
['08/18', '025739', '156', '6154.0', '16853.0', '19', '3']
['08/18', '025361', '157', '3333.0', '9931.0', '14', '1']
['08/18', '025361', '158', '513.0', '-902.0', '0', '0']
['08/18',

['08/18', '025553', '282', '20256.0', '-5115.0', '10', '3']
['08/18', '025553', '283', '12051.0', '12339.0', '20', '4']
['08/18', '025553', '284', '18974.0', '8126.0', '22', '4']
['08/18', '025553', '285', '13590.0', '-1504.0', '8', '1']
['08/18', '025553', '286', '12821.0', '-902.0', '9', '4']
['08/18', '025553', '287', '13590.0', '5719.0', '14', '3']
['08/18', '025553', '288', '8462.0', '2408.0', '8', '2']
['08/18', '025557', '289', '7179.0', '-3911.0', '2', '2']
['08/18', '025557', '290', '2821.0', '3311.0', '6', '1']
['08/18', '025557', '291', '6410.0', '-5717.0', '0', '0']
['08/18', '025557', '292', '5897.0', '-5115.0', '0', '0']
['08/18', '025557', '293', '4872.0', '-2406.0', '2', '2']
['08/18', '025489', '294', '28205.0', '603.0', '12', '5']
['08/18', '025489', '295', '6154.0', '-2707.0', '1', '1']
['08/18', '025489', '296', '18718.0', '-902.0', '7', '2']
['08/18', '025489', '297', '17692.0', '-3911.0', '5', '2']
['08/18', '025834', '298', '3846.0', '-3309.0', '0', '0']
['08/18'

['08/18', '025839', '422', '6410.0', '1506.0', '9', '1']
['08/18', '025873', '423', '8462.0', '-1203.0', '5', '1']
['08/18', '025873', '424', '17179.0', '-15045.0', '1', '1']
['08/18', '025643', '425', '7179.0', '-902.0', '4', '3']
['08/18', '025643', '426', '5897.0', '-5115.0', '0', '0']
['08/18', '025483', '427', '20000.0', '-7221.0', '10', '3']
['08/18', '025483', '428', '1282.0', '-902.0', '1', '1']
['08/18', '025734', '429', '12308.0', '-9930.0', '0', '0']
['08/18', '025734', '430', '3846.0', '6019.0', '8', '1']
['08/18', '025884', '431', '17692.0', '10533.0', '11', '2']
['08/18', '025884', '432', '6667.0', '-902.0', '2', '1']
['08/18', '025837', '433', '16923.0', '8427.0', '16', '3']
['08/18', '025837', '434', '5128.0', '-3610.0', '1', '1']
['08/18', '025837', '435', '14103.0', '-11133.0', '1', '1']
['08/18', '025859', '436', '24103.0', '24075.0', '50', '9']
['08/18', '025859', '437', '5128.0', '-4814.0', '0', '0']
['08/18', '025859', '438', '9487.0', '302.0', '8', '3']
['08/18',

['08/18', '025677', '561', '13077.0', '904.0', '9', '3']
['08/18', '025677', '562', '14103.0', '15348.0', '21', '1']
['08/18', '025677', '563', '15128.0', '-7221.0', '4', '2']
['08/18', '025677', '564', '23846.0', '11135.0', '23', '2']
['08/18', '025677', '565', '28205.0', '-18055.0', '5', '5']
['08/18', '025677', '566', '20769.0', '-17453.0', '1', '1']
['08/18', '025677', '567', '38974.0', '42131.0', '55', '12']
['08/18', '025677', '568', '12821.0', '-1805.0', '7', '1']
['08/18', '025677', '569', '23846.0', '22570.0', '33', '4']
['08/18', '025677', '570', '12051.0', '-6018.0', '4', '2']
['08/18', '025677', '571', '35128.0', '-16851.0', '10', '4']
['08/18', '025677', '572', '24872.0', '3612.0', '17', '5']
['08/18', '025677', '573', '21538.0', '904.0', '13', '4']
['08/18', '025677', '574', '28205.0', '-18957.0', '4', '2']
['08/18', '025677', '575', '20513.0', '302.0', '13', '4']
['08/18', '025677', '576', '31538.0', '-23772.0', '3', '3']
['08/18', '025677', '577', '26923.0', '24376.0', 

...

['08/18', '120148', '1106', '6261', '-4259.0', '0', '13', '28']
['08/18', '120148', '1107', '8469', '-4954.0', '0', '14', '38']
['08/18', '120148', '1108', '6797', '-4861.0', '0', '15', '25']
['08/18', '120148', '1109', '9248', '-1065.0', '0', '15', '64']
['08/18', '120148', '1110', '9519', '-4954.0', '0', '19', '47']
['08/18', '120148', '1111', '6575', '-4954.0', '0', '13', '26']
['08/18', '120148', '1112', '8923', '1019.0', '0', '21', '74']
['08/18', '120148', '1113', '8321', '-4954.0', '0', '19', '29']
['08/18', '120145', '1114', '3110', '-2454.0', '0', '5', '14']
['08/18', '120145', '1115', '1501', '-741.0', '0', '4', '10']
['08/18', '120145', '1116', '0', '0', '0', '0', '--']
['08/18', '120145', '1117', '3503', '787.0', '0', '6', '38']
['08/18', '120145', '1118', '3595', '2778.0', '0', '6', '51']
['08/18', '120145', '1119', '0', '0', '0', '0', '--']
['08/18', '120145', '1120', '4439', '-694.0', '0', '11', '40']
['08/18', '120145', '1121', '612', '324.0', '0', '1', '8']
['08/18', '

['08/18', '120122', '1239', '3736', '1389.0', '49', '14', '--']
['08/18', '120122', '1240', '1070', '-231.0', '8', '2', '--']
['08/18', '120122', '1241', '321', '1898.0', '17', '1', '--']
['08/18', '120122', '1242', '856', '324.0', '11', '2', '--']
['08/18', '120122', '1243', '303', '1389.0', '16', '1', '--']
['08/18', '120122', '1244', '5298', '926.0', '65', '16', '--']
['08/18', '120122', '1245', '95', '0', '1', '1', '--']
['08/18', '120122', '1246', '1020', '-556.0', '4', '3', '--']
['08/18', '120122', '1247', '4277', '-1991.0', '34', '15', '--']
['08/18', '120122', '1248', '1178', '93.0', '14', '3', '--']
['08/18', '120122', '1249', '3883', '-1389.0', '32', '9', '--']
['08/18', '120073', '1250', '1929', '-556.0', '0', '6', '13']
['08/18', '120073', '1251', '1105', '-787.0', '0', '2', '5']
['08/18', '120073', '1252', '1280', '-1389.0', '0', '2', '3']
['08/18', '120073', '1253', '2562', '-1157.0', '0', '9', '12']
['08/18', '120073', '1254', '3500', '-2315.0', '0', '13', '16']
['08/18

['08/18', '120053', '1371', '724', '-278.0', '0', '1', '5']
['08/18', '120053', '1372', '4282', '-93.0', '0', '10', '59']
['08/18', '120053', '1373', '4572', '-2963.0', '0', '11', '36']
['08/18', '119987', '1374', '2617', '185.0', '0', '0', '30']
['08/18', '119987', '1375', '3005', '-972.0', '0', '0', '25']
['08/18', '119987', '1376', '0', '0', '0', '0', '--']
['08/18', '119987', '1377', '2664', '-324.0', '0', '0', '30']
['08/18', '120033', '1378', '5773', '1435.0', '24', '14', '--']
['08/18', '120033', '1379', '2338', '231.0', '8', '8', '--']
['08/18', '120033', '1380', '3371', '972.0', '13', '6', '--']
['08/18', '120033', '1381', '1848', '1111.0', '9', '2', '--']
['08/18', '120033', '1382', '6329', '-741.0', '19', '14', '--']
['08/18', '120033', '1383', '6288', '1852.0', '26', '10', '--']
['08/18', '120036', '1384', '1409', '417.0', '5', '7', '--']
['08/18', '120036', '1385', '2006', '-602.0', '3', '10', '--']
['08/18', '120088', '1386', '942', '1343.0', '4', '3', '13']
['08/18', '12

['08/18', '120104', '1503', '5784', '-139.0', '0', '8', '20']
['08/18', '120104', '1504', '5946', '-1528.0', '0', '9', '26']
['08/18', '120138', '1505', '5716', '-2639.0', '0', '11', '24']
['08/18', '120138', '1506', '6490', '-4907.0', '0', '9', '13']
['08/18', '120138', '1507', '6553', '-324.0', '0', '11', '29']
['08/18', '120103', '1508', '6343', '-93.0', '38', '3', '21']
['08/18', '120103', '1509', '5122', '1620.0', '64', '5', '20']
['08/18', '120103', '1510', '6655', '-4167.0', '23', '7', '21']
['08/18', '120128', '1511', '6382', '93.0', '0', '1', '21']
['08/18', '120128', '1512', '3218', '-1389.0', '0', '0', '8']
['08/18', '120128', '1513', '6045', '-833.0', '0', '2', '15']
['08/18', '120128', '1514', '5804', '-463.0', '0', '1', '16']
['08/18', '120128', '1515', '4816', '2315.0', '0', '0', '18']


{'spreadsheetId': '1ch9Ckr3IiaMb8JLWNc-PWMEE59l4LbkU6ydfPI6mLAA',
 'updates': {'spreadsheetId': '1ch9Ckr3IiaMb8JLWNc-PWMEE59l4LbkU6ydfPI6mLAA',
  'updatedRange': 'S2RAW!F5025:M5048',
  'updatedRows': 24,
  'updatedColumns': 8,
  'updatedCells': 192}}